In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's PaLM 2](https://ai.google/discover/palm2/) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use PaLM2TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt GeminiTextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id in gcloud
#! gcloud config set project {PROJECT_ID}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [3]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [4]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [5]:
import bigframes.pandas as bf

# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bf.options.bigquery.project = PROJECT_ID

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [6]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [7]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.peek(n=5) # View an arbitrary five complaints

,consumer_complaint_narrative
24,"I sent disputed to Transunion, XXXX and XXXX f..."
942,"on XX/XX/2017 I sent XXXX, transunion, XXXX pr..."
1193,"On Wednesday, XXXX XXXX , I initiated a wir..."
1292,"Dear Sir or Madam, I am a victim of identity t..."
1377,"For the purpose of this complaint, I will refe..."


Downsample DataFrame to 10,000 records for model training.

In [8]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [9]:
from bigframes.ml.llm import PaLM2TextEmbeddingGenerator

model = PaLM2TextEmbeddingGenerator() # No connection id needed

In [10]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings

,text_embedding,statistics,ml_embed_text_status,content
545,[ 1.82510037e-02 -1.27867460e-02 -1.57095697e-...,"{""token_count"":178,""truncated"":false}",,My payments have been approximately {$89.00} w...
614,[ 5.40032536e-02 -5.28502129e-02 -5.33268750e-...,"{""token_count"":399,""truncated"":false}",,"Hi, I have contacted Trans Union XXXX XXXX abo..."
1236,[-5.32836001e-03 -5.84292673e-02 -5.86670786e-...,"{""token_count"":129,""truncated"":false}",,I have a XXXX XXXX XXXX credit card on my Exp...
1477,[ 3.02605387e-02 -4.37121317e-02 -2.70802993e-...,"{""token_count"":16,""truncated"":false}",,"Wrongs information, selling my information to ..."
2261,[ 2.35723313e-02 -3.73509154e-02 -6.44604117e-...,"{""token_count"":33,""truncated"":false}",,Please investigate and delete disputed item th...
2361,[ 1.04440488e-02 -9.37070698e-03 -7.36323372e-...,"{""token_count"":45,""truncated"":false}",,By the provisions of the Fair Credit Reporting...
2378,[ 3.04989032e-02 -4.08191867e-02 -6.18648790e-...,"{""token_count"":892,""truncated"":false}",,Since XX/XX/XXXX I have been trying to dispute...
3133,[ 0.00152804 -0.04189068 -0.04220504 -0.053740...,"{""token_count"":90,""truncated"":false}",,Out of the blue I received a debt collection n...
3140,[ 3.11435573e-02 -4.44000624e-02 -2.10917685e-...,"{""token_count"":372,""truncated"":false}",,My wife and I have been sending money to XXXX ...
3322,[ 2.75927987e-02 -6.23729872e-03 -3.83295454e-...,"{""token_count"":36,""truncated"":false}",,Phone calls from Convergent Outsourcing XXXX. ...


The model may have encountered errors while calculating embeddings for some rows. Filter out the errored rows before training the model. Alternatively, select these rows and retry the embeddings.

In [11]:
successful_rows = (
    (predicted_embeddings["ml_generate_embedding_status"] == "")
    # Series.str.len() gives the length of an array.
    # See: https://stackoverflow.com/a/41340543/101923
    & (predicted_embeddings["ml_generate_embedding_result"].str.len() != 0)
)
predicted_embeddings = predicted_embeddings[successful_rows]
predicted_embeddings


,text_embedding,statistics,ml_embed_text_status,content
545,[ 1.82510037e-02 -1.27867460e-02 -1.57095697e-...,"{""token_count"":178,""truncated"":false}",,My payments have been approximately {$89.00} w...
614,[ 5.40032536e-02 -5.28502129e-02 -5.33268750e-...,"{""token_count"":399,""truncated"":false}",,"Hi, I have contacted Trans Union XXXX XXXX abo..."
1236,[-5.32836001e-03 -5.84292673e-02 -5.86670786e-...,"{""token_count"":129,""truncated"":false}",,I have a XXXX XXXX XXXX credit card on my Exp...
1477,[ 3.02605387e-02 -4.37121317e-02 -2.70802993e-...,"{""token_count"":16,""truncated"":false}",,"Wrongs information, selling my information to ..."
2261,[ 2.35723313e-02 -3.73509154e-02 -6.44604117e-...,"{""token_count"":33,""truncated"":false}",,Please investigate and delete disputed item th...
2361,[ 1.04440488e-02 -9.37070698e-03 -7.36323372e-...,"{""token_count"":45,""truncated"":false}",,By the provisions of the Fair Credit Reporting...
2378,[ 3.04989032e-02 -4.08191867e-02 -6.18648790e-...,"{""token_count"":892,""truncated"":false}",,Since XX/XX/XXXX I have been trying to dispute...
3133,[ 0.00152804 -0.04189068 -0.04220504 -0.053740...,"{""token_count"":90,""truncated"":false}",,Out of the blue I received a debt collection n...
3140,[ 3.11435573e-02 -4.44000624e-02 -2.10917685e-...,"{""token_count"":372,""truncated"":false}",,My wife and I have been sending money to XXXX ...
3322,[ 2.75927987e-02 -6.23729872e-03 -3.83295454e-...,"{""token_count"":36,""truncated"":false}",,Phone calls from Convergent Outsourcing XXXX. ...


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [12]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [13]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["ml_generate_embedding_result"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.peek(n=5)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,text_embedding,statistics,ml_embed_text_status,content
182250,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.570560301900...",[ 4.70298417e-02 -4.08669300e-02 -2.99868709e-...,"{""token_count"":10,""truncated"":false}",,These are not my accounts. Please remove them.
3023485,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.523572693768...",[ 1.55437263e-02 -1.93240177e-02 -2.48466972e-...,"{""token_count"":10,""truncated"":false}",,This debt is not mine due to identity theft.
407254,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.515173566816...",[-0.01293471 -0.01959546 -0.02238463 -0.066214...,"{""token_count"":10,""truncated"":false}",,I do not owe this company money!!!!!
1509454,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.645342721754...",[ 3.21860723e-02 -2.67103072e-02 -4.78175096e-...,"{""token_count"":10,""truncated"":false}",,VIOLATES HIPPA AND CRA
2357848,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.519872186251...",[-1.88122243e-02 -2.68064123e-02 -4.69480827e-...,"{""token_count"":10,""truncated"":false}",,Receive numerous phone calls. I have no debt.


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use Gemini to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt GeminiTextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [14]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [15]:
# Build plain-text prompts to send to PaLM 2. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. Wrongs information, selling my information to third party. Incorrect reporting
2. I TIED TO BUY CAR AT XXXX, THEY GOT APPROVAL FROM XXXX  XXXX XXXX XXXX   WHICH ENDED A A LIE. THEY ALSO, PULLED MANY OTHERS I DID NT AGREED TO. SOLF PULLS ONLY
3. XX/XX/XXXX and XX/XX/XXXX inaccurate information reported 30 days late.
4. Im working on myCredit and I see a couple of inquiries that I have no idea where they came from.
5. I request a copy of all dispute results and documentary evidence from XXXX, and XXXX mailed to me

comment list 2:
1. My wife and I have been sending money to XXXX via my brother-in-law to finish a building project we have been working on since XXXX with target date of completion by XX/XX/XXXX. In XXXX XXXX my brother-in-law in was contacted by his bank to confirm he was not defrauding my wife. My brother-in-law confirmed he was helping to handle the building project by organizing and paying the workers. In XXXX XXXX Bank of America reach out to my wife t

In [16]:
# The plain English request we will make of PaLM 2
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. Wrongs information, selling my information to third party. Incorrect reporting
2. I TIED TO BUY CAR AT XXXX, THEY GOT APPROVAL FROM XXXX  XXXX XXXX XXXX   WHICH ENDED A A LIE. THEY ALSO, PULLED MANY OTHERS I DID NT AGREED TO. SOLF PULLS ONLY
3. XX/XX/XXXX and XX/XX/XXXX inaccurate information reported 30 days late.
4. Im working on myCredit and I see a couple of inquiries that I have no idea where they came from.
5. I request a copy of all dispute results and documentary evidence from XXXX, and XXXX mailed to me
comment list 2:
1. My wife and I have been sending money to XXXX via my brother-in-law to finish a building project we have been working on since XXXX with target date of completion by XX/XX/XXXX. In XXXX XXXX my brother-in-law in was contacted by his bank to confirm he was not defrauding my wife. My brother-in-law confirmed he was helping to handle the building project by organiz

Get a response from Gemini by making a call to Vertex AI using our connection.

In [17]:
from bigframes.ml.llm import GeminiTextGenerator

q_a_model = GeminiTextGenerator()

In [18]:
# Make a DataFrame containing only a single row with our prompt for Gemini
df = bf.DataFrame({"prompt": [prompt]})

In [19]:
# Send the request for Gemini to generate a response to our prompt
major_difference = q_a_model.predict(df)
# PaLM 2's response is the only row in the dataframe result 
major_difference["ml_generate_text_llm_result"].iloc[0]

'The most obvious difference between the two lists of comments is that list 1 is related to credit reporting disputes and list 2 is a collection of general consumer banking complaints.'

We now see GeminiTextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).